In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import onset_calculations as onset_calc
import matplotlib.pyplot as plt

In [ ]:
OnsetProcessor = onset_calc.OnsetProcessor()

def read_data(select_idx, mode):
    # Feel free to customize this function as per your needs
    
    filelist = os.listdir(f"./Dataset_V2")
    filename = filelist[select_idx]
        
    pickle_path = f'./motion_data_pkl/{filename.replace("_Dancers.csv", "_T.pkl")}'
        
    if os.path.isfile(pickle_path):
        with open(pickle_path, 'rb') as file:
            motion_data = pickle.load(file)

    fname = filename.replace("_Dancers.csv", "_T")
    onset_filename = filename.replace("_Dancers.csv", ".csv")
    danceanno_filename = filename
    mcycle_filename = filename.replace("_Dancers.csv", "_C.csv")

    drum_onsets_path = f"./drum_onsets/{onset_filename}"
    dance_anno_path = f"./Dataset_V2/{danceanno_filename}"
    mcycle_path = f"./virtual_cycles/{mcycle_filename}"

    # Load Djembe drum onsets
    drum_df = pd.read_csv(drum_onsets_path)
    # column_b = drum_df["J2"].dropna()
    # drum_onsets = column_b.to_numpy()

    # Load dance annotations
    dance_annotation_df = pd.read_csv(dance_anno_path)
    category_df = dance_annotation_df.groupby('mocap')
    category_df = category_df.get_group(mode)
    category_df = category_df.reset_index(drop=True)

    # Load meter cycle onsets
    df_metric = pd.read_csv(mcycle_path)
    loaded_mcycle_onsets = df_metric["Virtual Onset"].to_numpy()
    bpm_values = np.round(df_metric["bpm"].to_numpy(), 3)

    section_data = OnsetProcessor.onset_calculations(category_df, loaded_mcycle_onsets)


    return fname, motion_data, drum_df, category_df, df_metric, section_data, bpm_values

### Read data

In [ ]:
select_idx = 3      # Choose any idx number (of 47) to access a piece from the file list
mode = "in"         # in: individual dance, gr: group dance, au: dancer in audience
fname, motion_data, drum_df, dance_annotation_df, metric_df, section_data, bpm_values = read_data(select_idx, mode)

### Djembe drum onsets

In [ ]:
# Djembe drum onsets: DunDun, Djembe1, Djembe 2
drum_df.head()

### Dance annotation

In [ ]:
# Each row is termed as a "Section",
dance_annotation_df

### Metric cycle

In [ ]:
metric_df.head()

### Metadata and onset data

In [ ]:
# This part extracts relevannt data from the dance_annotation_df and metric_df for a section in dance_annotation_df


section_idx = 2  # Replace with the index you need from the dance_annotation_df table

section = section_data[section_idx]
section_name, section_info = list(section.items())[0]

# Access metadata and onset data
section_meta_data = section_info["section_meta_data"]
section_onset_data = section_info["section_onset_data"]

# Unpack metadata
start_timestamp = section_meta_data["start_timestamp"]
end_timestamp = section_meta_data["end_timestamp"]
category = section_meta_data["category"]
start_sec = section_meta_data["start"]
end_sec = section_meta_data["end"]
duration_sec = section_meta_data["duration"]

# Unpack onset data
cycle_onsets = section_onset_data["cycle_onsets"]
total_cycles = section_onset_data["total_blocks"]
cycle_period_list = section_onset_data["cycle_period_list"]
all_window_onsets = section_onset_data["all_window_onsets"]
window_period_list = section_onset_data["window_period_list"]

section_start_idx = np.where(np.isclose(metric_df["Virtual Onset"], cycle_onsets[0], atol=1e-6))[0][0]     # use this index to pull values from other columns (ioi etc)
section_end_idx = np.where(np.isclose(metric_df["Virtual Onset"], cycle_onsets[-1], atol=1e-6))[0][0]      # use this index to pull values from other columns (ioi etc)
section_bpm = metric_df.iloc[section_start_idx+1:section_end_idx+1]["bpm"].to_numpy()          # first bpm value --> cycle_onset[n] & cycle_onsets[n-1]

print("Piece:", fname)
print(f"Section: {section_name}")
print(f"Start (sec): {start_sec}")
print(f"End (sec): {end_sec}")
print(f"Category: {category}")
print(f"Duration (sec): {duration_sec}\n")

print(f"Cycle Onsets: {cycle_onsets}\n")
print(f"Cycle (start, end): {all_window_onsets}\n")    # (start, end) onset of each cycle in original time (seconds)
print(f"Cycle Period List: {cycle_period_list}\n")     # period = end-start
print(f"BPM: {section_bpm}\n")     # period = end-start
print(f"Total Cycles: {total_cycles}\n")

### Motion Data

In [ ]:
# Segment Names:
# 'SEGMENT_PELVIS', 'SEGMENT_L5', 'SEGMENT_L3', 'SEGMENT_T12', 'SEGMENT_T8', 
# 'SEGMENT_NECK', 'SEGMENT_HEAD', 'SEGMENT_RIGHT_SHOULDER', 'SEGMENT_RIGHT_UPPER_ARM', 
# 'SEGMENT_RIGHT_FOREARM', 'SEGMENT_RIGHT_HAND', 'SEGMENT_LEFT_SHOULDER', 'SEGMENT_LEFT_UPPER_ARM', 
# 'SEGMENT_LEFT_FOREARM', 'SEGMENT_LEFT_HAND', 'SEGMENT_RIGHT_UPPER_LEG', 'SEGMENT_RIGHT_LOWER_LEG', 
# 'SEGMENT_RIGHT_FOOT', 'SEGMENT_RIGHT_TOE', 'SEGMENT_LEFT_UPPER_LEG', 'SEGMENT_LEFT_LOWER_LEG', 
# 'SEGMENT_LEFT_FOOT', 'SEGMENT_LEFT_TOE'

mocap_fps = 240
start_f = np.round(start_sec*mocap_fps).astype(int)     # start frame of the current section
end_f = np.round(end_sec*mocap_fps).astype(int)         # end frame frame of the current section


segment_name = 'SEGMENT_RIGHT_HAND'

# Accessing segment data for the selected section in previous step
position_data = motion_data['position'][segment_name][start_f:end_f,:]                                  # numpy array, size (n,3)
velocity_data = motion_data['velocity'][segment_name][start_f:end_f,:]                                  # numpy array, size (n,3)  
acceleration_data = motion_data['acceleration'][segment_name][start_f:end_f,:]                          # numpy array, size (n,3)
orientation_data = motion_data['orientation'][segment_name][start_f:end_f,:]                          # numpy array, size (n,3)
angular_velocity_data = motion_data['angular_velocity'][segment_name][start_f:end_f,:]                # numpy array, size (n,3)
angular_accelaration_data = motion_data['angular_accelaration'][segment_name][start_f:end_f,:]        # numpy array, size (n,3)
sensor_orientation = motion_data['sensor_orientation'][segment_name][start_f:end_f,:]                 # numpy array, size (n,3)

# Accessing axes data
position_data_X = motion_data['velocity'][segment_name][:,0]    # X-axis, 1d numpy array   
position_data_Y = motion_data['position'][segment_name][:,1]    # Y-axis
position_data_Z = motion_data['position'][segment_name][:,2]    # Z-axis

time_axis = np.arange(len(position_data))/mocap_fps     # for plotting

In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(position_data_X[start_f:end_f] )

### Load CSV file for the selected piece

In [ ]:
# Example code snippet to load csv files from the results folder

# Change path and filename here to access different onset csv file
feet_onsets_path = f"./extracted_data/{fname}/onset_info/{fname}_both_feet_onsets.csv"
hand_onsets_path = f"./extracted_data/{fname}/onset_info/{fname}_both_hand_onsets.csv"

feet_onset_df = pd.read_csv(feet_onsets_path, usecols=[0])      # onsets from both feet combined
hand_onset_df = pd.read_csv(hand_onsets_path, usecols=[0])      # # onsets from both hand combined

print("Extracted onsets for section", section_idx)
feet_onset_df

### Normalized onsets per cycles for all the sections

In [ ]:
# Loading pickle files from the modes sub folder inside the main piece folder

p= f"./extracted_data/{fname}/{mode}/{fname}_Both_Feet_Onsets_section_cycles.pkl"
with open(p, 'rb') as file:
    c_data = pickle.load(file)

c_data[1]
# Each pickle file when loaded is a list that contains a list of arrays for each section, with each array representing the normalized onsets for a cycle.
# c_data[0] is for section 1
# c_data[1] is for section 2, and so on

### Calculate beat onsets

In [ ]:
def calculate_beat_onsets(cycle_onsets):
        beat_ref = []
        for i in range(len(cycle_onsets) - 1):
            start = cycle_onsets[i]
            end = cycle_onsets[i + 1]
            # Generate beat onsets by dividing the interval into 4 equal parts
            beat_onsets = np.linspace(start, end, num=5)[:-1]
            beat_ref.extend(beat_onsets)
        return np.round(np.array(beat_ref), 3)
    
beat_ref = calculate_beat_onsets(cycle_onsets)
beat_ref
(253.751, 254.783)